In [1]:
import os
from dotenv import load_dotenv
import openai
import csv
import pandas as pd
from openai import OpenAI
import json

In [2]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("No OpenAI API key found. Check your .env file.")

In [3]:
openai.api_key = openai_api_key

In [4]:
client = openai.OpenAI()
def get_completion(messages, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

In [5]:
instruction = f"""
You will be provided with a set of IMDB movie reviews an their sentiment \
The sentiment is either positive or negative.\
Your task is to identify the sentiment of the review.\
To do this, you first work out your own solution and determine whether the review is positive or negative.\
Then compare your soulution with the provided solution.\
After that you evaluate whether your solution matches the provides solution.\
Do not look at the provided solution before you have worked out your own solution.\
Use the following scale to evaluate your solution:\
match, partial match, no match\
Output the result in JSON format with the following keys:\
review, your_solution, provided_solution, evaluation
"""


In [6]:
df = pd.read_csv("IMDB Dataset.csv")

In [8]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [32]:
results = []

In [40]:
for index, row in df.iloc[:5].iterrows():
    # Format the review and sentiment into a single string
    combined_input = f"Review: {row['review']} Sentiment: {row['sentiment']}"

In [41]:
 messages = [
        {"role": "system", "content": instruction},
        {"role": "user", "content": combined_input}
    ]

In [45]:
response_text = get_completion(messages)

print(response_text)

{
    "review": "Petter Mattei's \"Love in the Time of Money\" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different situations we encounter. <br /><br />This being a variation on the Arthur Schnitzler's play about the same theme, the director transfers the action to the present time New York where all these different characters meet and connect. Each one is connected in one way, or another to the next person, but no one seems to know the previous point of contact. Stylishly, the film has a sophisticated luxurious look. We are taken to see how these people live and the world they live in their own habitat.<br /><br />The only thing one gets out of all these souls in the picture is the different stages of loneliness each one inhabits. A big city is not exactly the best place in which human relations find sincere fulfillment, as one discerns

In [46]:
import openai

# Initialize the OpenAI client
client = openai.OpenAI()

def get_completion(review, sentiment, model="gpt-3.5-turbo"):
    # Format the input message including review and sentiment
    input_message = f"Review: {review} Sentiment: {sentiment}"
    
    # Create the messages payload
    messages = [
        {"role": "system", "content": instruction},
        {"role": "user", "content": input_message}
    ]
    
    # Make the API call
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    
    # Return the message content from the response
    return response.choices[0].message.content

# Assuming 'df' is your DataFrame and 'instruction_text' is defined
results = []
for index, row in df.iloc[:5].iterrows():
    # Call the modified get_completion function with review and sentiment
    response_content = get_completion(row['review'], row['sentiment'])
    
    # Store the response
    results.append({
        "index": index,
        "response": response_content
    })

# Print or process the results
for result in results:
    print(f"Response for row {result['index']}: {result['response']}")


Response for row 0: {
    "review": "One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main

In [49]:
response_data = json.loads(response_text)

In [50]:
responses.append(response_data)

In [51]:
df_responses = pd.DataFrame(responses)

In [53]:
df_responses.head()

,review,response,sentiment,your_solution,provided_solution,evaluation
0,"Petter Mattei's ""Love in the Time of Money"" is...","{\n ""review"": ""Petter Mattei's \""Love in th...",positive,NaN,NaN,NaN
1,"Petter Mattei's ""Love in the Time of Money"" is...","{\n ""review"": ""Petter Mattei's \""Love in th...",positive,NaN,NaN,NaN
2,"Petter Mattei's ""Love in the Time of Money"" is...",NaN,NaN,positive,positive,match
